In [ ]:
import analizer as an

# Data import
df_origin = pd.read_excel('report.xls')

df_orders = an.extend_with_grid_details(df_origin)
df_grids = an.get_grids(df_orders)
df_sum an.get_summary(df_orders=df_orders,
                      df_grids=df_grids)
